In [ ]:
!nvidia-smi

Sun Jan  2 20:02:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Load data

In [ ]:
import scipy
import librosa
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
import scipy.io.wavfile
import time
import IPython
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import Subset
import json

In [ ]:
gt = pd.read_csv('/content/drive/MyDrive/COSRMAL_CHALLENGE/train.csv')
gt.head()

,id,container id,scenario,background,illumination,width at the top,width at the bottom,height,depth,container capacity,container mass,filling type,filling level,filling density,filling mass,object mass,handover starting frame,handover start timestamp,handover hand,action,nframes,folder_num,file_name,num,subject,filling_type,filling_level,back,light,camera_id,start,end
0,0,2,2,1,0,69.0,42.0,72.0,-1.0,185.000,2.0,2,1,0.82,76.0,78.0,-1,-1,-1,1.0,291576,2,s2_fi2_fu1_b1_l0,70,2,2,1,1,0,2,0.75,3.5
1,1,7,0,0,0,193.0,193.0,241.0,69.0,3209.397,59.0,0,0,0.00,0.0,59.0,-1,-1,-1,0.0,118483,7,s0_fi0_fu0_b0_l0,0,0,0,0,0,0,2,-1.00,-1.0
2,2,2,0,1,0,69.0,42.0,72.0,-1.0,185.000,2.0,3,1,1.00,93.0,95.0,-1,-1,-1,1.0,572008,2,s0_fi3_fu1_b1_l0,22,0,3,1,1,0,2,3.40,6.5
3,3,8,0,1,0,135.0,135.0,164.0,56.0,1239.840,31.0,0,0,0.00,0.0,31.0,-1,-1,-1,0.0,141680,8,s0_fi0_fu0_b1_l0,2,0,0,0,1,0,2,-1.00,-1.0
4,4,4,1,1,0,88.0,56.0,91.0,-1.0,296.000,86.0,1,1,0.34,45.0,131.0,-1,-1,-1,1.0,138681,4,s1_fi1_fu1_b1_l0,34,1,1,1,1,0,2,0.75,1.8


In [ ]:
class AudioProcessing():
    def __init__(self,sample_rate,signal,frame_length_t=0.025,frame_stride_t=0.01,nfilt =64):
        
        self.sample_rate=sample_rate
        self.signal = signal
        self.frame_length_t=frame_length_t
        self.frame_stride_t=frame_stride_t
        self.signal_length_t=float(signal.shape[0]/sample_rate)
        self.frame_length=int(round(frame_length_t * sample_rate)) #number of samples
        self.frame_step=int(round(frame_stride_t * sample_rate))
        self.signal_length = signal.shape[0]
        self.nfilt=nfilt
        self.num_frames = int(np.ceil(float(np.abs(self.signal_length - self.frame_length)) / self.frame_step))
        self.pad_signal_length=self.num_frames * self.frame_step + self.frame_length
        self.NFFT=512
        
    def cal_frames(self):
        z = np.zeros([self.pad_signal_length - self.signal_length,8])
        pad_signal = np.concatenate([self.signal, z], 0)
        indices = np.tile(np.arange(0, self.frame_length), (self.num_frames, 1)) + np.tile(np.arange(0, self.num_frames * self.frame_step, self.frame_step), (self.frame_length, 1)).T
        frames = pad_signal[indices.astype(np.int32, copy=False)]
        return frames
        
    def calc_MFCC(self):
        # 291576
        pre_emphasis=0.97

        # (n,8)
        emphasized_signal=np.concatenate([self.signal[0,:].reshape([1,-1]),  self.signal[1:,:] - pre_emphasis * self.signal[:-1,:]], 0)
        z = np.zeros([self.pad_signal_length - self.signal_length,8])
        pad_signal = np.concatenate([emphasized_signal, z], 0)
        indices = np.tile(np.arange(0, self.frame_length), (self.num_frames, 1)) + np.tile(np.arange(0, self.num_frames * self.frame_step, self.frame_step), (self.frame_length, 1)).T
        frames = pad_signal[indices.astype(np.int32, copy=False)]
        frames=frames*np.hamming(self.frame_length).reshape(1,-1,1)
        frames=frames.transpose(0,2,1)
        mag_frames = np.absolute(np.fft.rfft(frames,self.NFFT))
        pow_frames = ((1.0 / self.NFFT) * ((mag_frames) ** 2))
        filter_banks = np.dot(pow_frames, self.cal_fbank().T)
        filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)
        filter_banks = 20 * np.log10(filter_banks)  # dB
        filter_banks =filter_banks.transpose(0,2,1)
        
        return filter_banks
           
    def cal_fbank(self):
        
        low_freq_mel = 0
        high_freq_mel = (2595 * np.log10(1 + (self.sample_rate / 2) / 700))  
        mel_points = np.linspace(low_freq_mel, high_freq_mel, self.nfilt + 2)  
        hz_points = (700 * (10**(mel_points / 2595) - 1)) 
        bin = np.floor((self.NFFT + 1) * hz_points / self.sample_rate)
        fbank = np.zeros((self.nfilt, int(np.floor(self.NFFT / 2 + 1))))
        for m in range(1, self.nfilt + 1):
            f_m_minus = int(bin[m - 1])   # left
            f_m = int(bin[m])             # center
            f_m_plus = int(bin[m + 1])    # right

            for k in range(f_m_minus, f_m):
                fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
            for k in range(f_m, f_m_plus):
                fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])
        return fbank

# Task 2

### Preprocess

In [ ]:
# base_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/'
# audio_folder = '/content/drive/MyDrive/COSRMAL_CHALLENGE/train/audio/'
# os.makedirs(os.path.join(base_path, 'audios'), exist_ok=True)
# mfcc_path = (os.path.join(base_path, 'audios', 'mfcc'))
# raw_path = (os.path.join(base_path, 'audios', 'raw'))
# os.makedirs(mfcc_path,exist_ok=True)
# os.makedirs(raw_path,exist_ok=True)

# audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
# save_size=64
# ratio_step = 0.25
# count = 0
# pouring_or_shaking_list = []
# file_idx_list = []
# filling_type_list = []
# pbar = tqdm(total=len(audio_paths))

# for i, path in enumerate(audio_paths):
#   id = i
#   start_time = gt[gt.id==id]['start'].item()
#   end_time = gt[gt.id==id]['end'].item()
#   filling_type = gt[gt.id==id]['filling_type'].item()
#   sample_rate, signal = scipy.io.wavfile.read(path)
#   ap = AudioProcessing(sample_rate,signal,nfilt=save_size)
#   mfcc = ap.calc_MFCC()
#   raw_frames = ap.cal_frames()
#   mfcc_length=mfcc.shape[0]

#   if mfcc_length < save_size:
#     print("file {} is too short".format(id))
#   else:
#     f_step=int(mfcc.shape[1]*ratio_step)
#     f_length=mfcc.shape[1]
#     save_mfcc_num=int(np.ceil(float(np.abs(mfcc_length - save_size)) / f_step))

#     for i in range(save_mfcc_num):
#       count += 1
#       tmp_mfcc = mfcc[i*f_step:save_size+i*f_step,: ,:]
#       if start_time == -1:
#           pouring_or_shaking_list.append(0)
#       elif start_time/ap.signal_length_t*mfcc_length<i*f_step+f_length*0.75 and end_time/ap.signal_length_t*mfcc_length>i*f_step+f_length*0.25:
#           pouring_or_shaking_list.append(1) 
#       else:
#           pouring_or_shaking_list.append(0)
      
#       filling_type_list.append(filling_type)
#       file_idx_list.append(id)

#       np.save(os.path.join(mfcc_path, "{0:06d}".format(count)), tmp_mfcc)
#   pbar.update()


# np.save(os.path.join(base_path, 'audios', 'pouring_or_shaking'), np.array(pouring_or_shaking_list) )
# np.save(os.path.join(base_path, 'audios', 'filling_type'), np.array(filling_type_list))


In [ ]:
base_path = '/content/'
audio_folder = '/content/drive/MyDrive/COSRMAL_CHALLENGE/train/audio/'
os.makedirs(os.path.join(base_path, 'audios'), exist_ok=True)
mfcc_path = (os.path.join(base_path, 'audios', 'mfcc'))
raw_path = (os.path.join(base_path, 'audios', 'raw'))
os.makedirs(mfcc_path,exist_ok=True)
os.makedirs(raw_path,exist_ok=True)

audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
save_size=64
ratio_step = 0.25
count = 0
pouring_or_shaking_list = []
file_idx_list = []
filling_type_list = []
pbar = tqdm(total=len(audio_paths))

for i, path in enumerate(audio_paths):
  id = i
  start_time = gt[gt.id==id]['start'].item()
  end_time = gt[gt.id==id]['end'].item()
  filling_type = gt[gt.id==id]['filling_type'].item()
  sample_rate, signal = scipy.io.wavfile.read(path)
  ap = AudioProcessing(sample_rate,signal,nfilt=save_size)
  mfcc = ap.calc_MFCC()
  mfcc_length=mfcc.shape[0]

  if mfcc_length < save_size:
    print("file {} is too short".format(id))
  else:
    f_step=int(mfcc.shape[1]*ratio_step)
    f_length=mfcc.shape[1]
    save_mfcc_num=int(np.ceil(float(np.abs(mfcc_length - save_size)) / f_step))

    for i in range(save_mfcc_num):
      count += 1
      tmp_mfcc = mfcc[i*f_step:save_size+i*f_step,: ,:]
      if start_time == -1:
          pouring_or_shaking_list.append(0)
      elif start_time/ap.signal_length_t*mfcc_length<i*f_step+f_length*0.75 and end_time/ap.signal_length_t*mfcc_length>i*f_step+f_length*0.25:
          pouring_or_shaking_list.append(1) 
      else:
          pouring_or_shaking_list.append(0)
      
      filling_type_list.append(filling_type)
      file_idx_list.append(id)

      np.save(os.path.join(mfcc_path, "{0:06d}".format(count)), tmp_mfcc)
  pbar.update()


np.save(os.path.join(base_path, 'audios', 'pouring_or_shaking'), np.array(pouring_or_shaking_list) )
np.save(os.path.join(base_path, 'audios', 'filling_type'), np.array(filling_type_list))


  0%|          | 0/684 [00:00<?, ?it/s]

In [ ]:
filling_type = np.load(os.path.join(base_path, 'audios', 'filling_type.npy'))
pouring_or_shaking = np.load(os.path.join(base_path,  'audios', 'pouring_or_shaking.npy'))

label = filling_type * pouring_or_shaking

In [ ]:
#from torchinfo import summary
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
#model = Net(8, 4).to(device)
#summary(model, input_size=(32, 8, 64, 64))

Using device: cuda


## Models

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv01 = nn.Conv2d(8, 32, 3,padding=1)#64
        self.conv02 = nn.Conv2d(32, 32, 3,padding=1)#64
        self.bn1=nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)#32

        self.conv03 = nn.Conv2d(32, 64, 3,padding=1)#32
        self.conv04 = nn.Conv2d(64, 64, 3,padding=1)#32
        self.bn2=nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)#16

        self.conv05 = nn.Conv2d(64, 128, 3,padding=1)#16
        self.conv06 = nn.Conv2d(128, 128, 3,padding=1)#16
        self.conv07 = nn.Conv2d(128, 128, 3,padding=1)#16
        self.bn3=nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)#8

        self.conv08 = nn.Conv2d(128, 256, 3,padding=1)#8
        self.conv09 = nn.Conv2d(256, 256, 3,padding=1)#8
        self.conv10 = nn.Conv2d(256, 256, 3,padding=1)#8
        self.bn4=nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2, 2)#4

        self.conv11 = nn.Conv2d(256, 256, 3,padding=1)#4
        self.conv12 = nn.Conv2d(256, 256, 3,padding=1)#4
        self.conv13 = nn.Conv2d(256, 256, 3,padding=1)#4
        self.bn5=nn.BatchNorm2d(256)
        self.pool5 = nn.MaxPool2d(2, 2)#2

       
        self.fc1 = nn.Linear(256 * 2 * 2, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 4)

        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        self.softmax = nn.Softmax(dim=1)



    def forward(self, x):
        x = F.relu(self.conv01(x))
        x = F.relu(self.conv02(x))
        x = self.pool1(self.bn1(x))

        x = F.relu(self.conv03(x))
        x = F.relu(self.conv04(x))
        x = self.pool2(self.bn2(x))

        x = F.relu(self.conv05(x))
        x = F.relu(self.conv06(x))
        x = F.relu(self.conv07(x))
        x = self.pool3(self.bn3(x))

        x = F.relu(self.conv08(x))
        x = F.relu(self.conv09(x))
        x = F.relu(self.conv10(x))
        x = self.pool4(self.bn4(x))

        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.relu(self.conv13(x))
        x = self.pool5(self.bn5(x))


        x = x.view(-1, 256 * 2 * 2)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        return self.softmax(x)   

## Datasets

In [ ]:
class audioDataSet(Dataset):
  def __init__(self,root_pth,test=False,transform = None):
    print("Dataset initializing...")
    class_num=4
    self.audio_pth = os.path.join(root_pth, 'audios', 'mfcc')
    filling_type = np.load(os.path.join(root_pth, 'audios', 'filling_type.npy'))
    pouring_or_shaking = np.load(os.path.join(root_pth,  'audios', 'pouring_or_shaking.npy'))
    self.label = filling_type * pouring_or_shaking
    self.is_test=test
    self.each_class_size = []
    for i in range(class_num):
        self.each_class_size.append(np.count_nonzero(self.label==i))
    # mx=0
    # mn=1000
    # for idx in tqdm(range(self.label.shape[0])):
    #   data=np.load(os.path.join(self.audio_pth, "{0:06d}".format(idx+1) + '.npy'), allow_pickle=True)
    #   tmp_max=np.max(data)
    #   tmp_min=np.min(data)
    #   if mx<tmp_max:
    #       mx=tmp_max
    #   if mn>tmp_min:
    #       mn=tmp_min
    mx=194.19187653405487
    mn=-313.07119549054045
    
    self.mn=mn
    self.mx=mx
  def __len__(self):
    return self.label.shape[0]
  def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        lbl = -1

        if self.is_test is False:
            lbl = self.label[idx]
        data=np.load(os.path.join(self.audio_pth, "{0:06d}".format(idx+1) + '.npy'), allow_pickle=True)
        data= (data-self.mn)/(self.mx-self.mn)
        data=data.transpose(2,0,1)
        data=torch.from_numpy(data.astype(np.float32))
        return data , lbl
            
  def get_each_class_size(self):
    return np.array(self.each_class_size)

In [ ]:
mydataset = audioDataSet(base_path)

Dataset initializing...


In [ ]:
print(mydataset.mn)
print(mydataset.mx)
print(len(mydataset))

-313.07119549054045
194.19187653405487
31812


## Training and Testing

In [ ]:
def train(model, train_loader, optimizer, criterion = nn.CrossEntropyLoss()):
  model.train()
  loss_train = 0.0
  correct_train = 0.0
  num_train = len(train_loader)
  for batch_idx, (audio, target) in enumerate(train_loader):
    audio = audio.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    outputs = model.forward(audio)

    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    loss_train += loss.item() / num_train
    _, preds=torch.max(outputs,1)
    correct_train+=torch.sum(preds==target).item()
  
  return loss_train, correct_train


def evaluate(model, testloader, criterion = nn.CrossEntropyLoss()):
  model.eval()
  loss_test = 0
  correct_test=0
  num_val = len(testloader)
  with torch.no_grad():
    for batch_idx, (audio, target) in enumerate(testloader):
      audio = audio.to(device)
      target = target.to(device)
      outputs = model.forward(audio)
      loss = criterion(outputs, target)
      loss_test += loss.item() / num_val
      _, preds=torch.max(outputs,1)
      correct_test+=torch.sum(preds==target).item()
  
  return loss_test, correct_test


In [ ]:
bs = 100
train_split = 0.8
lr = 1e-5
epochs = 200
n_samples = len(mydataset)
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = int(train_split * n_samples)
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(mydataset, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True)
for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("Epoch {}/{} train loss:{:.4f} train acc:{:.2f}% ".format(epoch+1,epochs, loss_train, 100 * correct_train/num_train))
  print("Epoch {}/{} val loss:{:.4f} val acc:{:.2f}% ".format(epoch+1,epochs, loss_val, 100 * correct_val/num_val))

  if loss_val < best_loss:
    best_loss = loss_val
    torch.save(model, os.path.join(base_path, 'audios', "best_loss.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model, os.path.join(base_path, 'audios', "best_val.pth"))
  




KeyboardInterrupt: ignored

In [ ]:
from mobile import MobileNetV3_Large

bs = 100
train_split = 0.8
lr = 1e-3
epochs = 200
n_samples = len(mydataset)
model = MobileNetV3_Large().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = int(train_split * n_samples)
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(mydataset, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True)
for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train, 100 * correct_train/num_train,
      loss_val, 100 * correct_val/num_val))

  if loss_val < best_loss:
    best_loss = loss_val
    torch.save(model, os.path.join(base_path, 'audios', "bl-mobile.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model, os.path.join(base_path, 'audios', "bv-mobile.pth"))
  




1/200 train loss:0.5775 train acc:79.35% val loss:0.5985 val acc:79.21%
2/200 train loss:0.3572 train acc:87.87% val loss:0.3697 val acc:86.89%
3/200 train loss:0.2895 train acc:89.89% val loss:0.3728 val acc:87.68%
4/200 train loss:0.2644 train acc:90.48% val loss:0.3346 val acc:88.31%
5/200 train loss:0.2458 train acc:91.26% val loss:0.5997 val acc:73.06%
6/200 train loss:0.2228 train acc:91.91% val loss:0.4331 val acc:88.59%
7/200 train loss:0.2042 train acc:92.81% val loss:0.2907 val acc:89.39%
8/200 train loss:0.1795 train acc:93.65% val loss:0.6188 val acc:78.47%
9/200 train loss:0.1554 train acc:94.29% val loss:0.2454 val acc:91.56%
10/200 train loss:0.1468 train acc:94.63% val loss:0.5624 val acc:80.92%
11/200 train loss:0.1265 train acc:95.43% val loss:0.3431 val acc:89.66%
12/200 train loss:0.1140 train acc:95.92% val loss:0.4471 val acc:88.10%
13/200 train loss:0.1017 train acc:96.37% val loss:0.3115 val acc:91.62%
14/200 train loss:0.0938 train acc:96.56% val loss:0.3609 va

In [ ]:
model_pretrained = torch.load(os.path.join(base_path, 'audios', "bv-mobile.pth"))
model_pretrained.to(device)
model_pretrained.eval()

voting_dir = '/content/drive/MyDrive/COSRMAL_CHALLENGE/audios'

mfcc_MAX_VALUE=194.19187653405487
mfcc_MIN_VALUE=-313.07119549054045

t2_MAX_VALUE = 57.464638
t2_MIN_VALUE = -1.1948369
save_size=64

start = time.time()

audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
save_data = {}
data_num = 0
for i, path in enumerate(audio_paths):
  count_pred = [0] * 4
  pred_list = []
  sample_rate, signal = scipy.io.wavfile.read(path)
  ap = AudioProcessing(sample_rate,signal)
  mfcc = ap.calc_MFCC()
  mfcc_length=mfcc.shape[0]
  f_step=int(mfcc.shape[1]*0.25)
  f_length=mfcc.shape[1]
  save_mfcc_num=int(np.ceil(float(np.abs(mfcc_length - save_size)) /f_step))
  for i in range(save_mfcc_num):
      tmp_mfcc = mfcc[i*f_step:save_size+i*f_step,: ,:]
      tmp_mfcc= (tmp_mfcc-mfcc_MIN_VALUE)/(mfcc_MAX_VALUE-mfcc_MIN_VALUE)
      tmp_mfcc=tmp_mfcc.transpose(2,0,1)
      audio=torch.from_numpy(tmp_mfcc.astype(np.float32))
      audio=torch.unsqueeze(audio, 0)
      audio = audio.to(device)

      with torch.no_grad():
        pred_T2 = model_pretrained.forward(audio)
        _,pred_T2=torch.max(pred_T2,1)
        count_pred[pred_T2.item()]+=1
        pred_list.append(pred_T2.item())
  if  count_pred[1]>5 or count_pred[2]>5 or count_pred[3]>5:
    final_pred_T2=count_pred[1:4].index(max(count_pred[1:4]))+1
  else:
    final_pred_T2=0
  
  file_name = path.split(os.path.sep)[-1].replace('.wav', '')
  #print("sequence:{}, frequency:{}".format(file_name, count_pred))
  to_save_data = {"data_num":data_num,
                  "file":file_name,
                  "count_pred":count_pred,
                  "final_pred":final_pred_T2,
                  'pred':pred_list}
  save_data["{}".format(file_name)] = to_save_data
  data_num+=1

with open (os.path.join(voting_dir, "voting.json"), 'w') as f:
  json.dump(save_data, f, indent=2, ensure_ascii=False)
  elapsed_time = time.time() - start
  print("elapsed_time:{}".format(elapsed_time) + "sec")



In [ ]:
f = open(os.path.join(voting_dir, "voting.json"))
vote_js = json.load(f)

vote = pd.DataFrame(vote_js).T
vote.head()

,data_num,file,count_pred,final_pred,pred
000000,0,000000,"[19, 0, 19, 0]",2,"[0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
000001,1,000001,"[13, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
000002,2,000002,"[56, 0, 0, 21]",3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
000003,3,000003,"[16, 0, 0, 0]",0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
000004,4,000004,"[7, 9, 0, 0]",1,"[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]"


In [ ]:
gt = pd.read_csv('/content/drive/MyDrive/COSRMAL_CHALLENGE/train.csv')
acc = np.sum(gt['filling_type'].to_numpy() == vote['final_pred'].to_numpy()) / len(gt['filling_type'])
print('Acc: {:.2f}%'.format(100 * acc))

Acc: 100.00%


### Efficient

In [ ]:
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""

import torch
import torch.nn as nn
import math

__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=1000, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(8, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def extract(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        feature = x.view(x.size(0), -1)
        x = self.classifier(feature)

        return feature, x


    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

### Efficient train

In [ ]:
my_save_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
bs = 100
train_split = 0.8
lr = 1e-4
epochs = 200
n_samples = len(mydataset)
model = effnetv2_xl().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = int(train_split * n_samples)
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(mydataset, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True,
                            num_workers=1)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True,
                          num_workers=1)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("{}/{} train loss:{:.4f} train acc:{:.2f}% val loss:{:.4f} val acc:{:.2f}%".format(
      epoch+1,epochs, loss_train, 100 * correct_train/num_train,
      loss_val, 100 * correct_val/num_val))
  

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "bl-efficient-xl.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model.state_dict(), os.path.join(my_save_path, 
                                              'audios', 
                                              'efficient',
                                              "XL-{:.2f}.pth".format(100 * correct_val/num_val)))
    
  




1/200 train loss:1.8104 train acc:69.01% val loss:0.8256 val acc:77.60%
2/200 train loss:0.6068 train acc:79.74% val loss:0.6122 val acc:77.26%
3/200 train loss:0.4565 train acc:84.31% val loss:0.3969 val acc:86.47%
4/200 train loss:0.3449 train acc:88.14% val loss:0.4365 val acc:84.02%
5/200 train loss:0.2854 train acc:89.94% val loss:0.3182 val acc:89.44%
6/200 train loss:0.2235 train acc:92.07% val loss:0.3550 val acc:89.74%
7/200 train loss:0.1681 train acc:94.10% val loss:0.4066 val acc:85.31%
8/200 train loss:0.1265 train acc:95.56% val loss:0.2367 val acc:92.22%
9/200 train loss:0.0840 train acc:97.08% val loss:0.2572 val acc:92.42%
10/200 train loss:0.0688 train acc:97.67% val loss:0.2259 val acc:93.59%
11/200 train loss:0.0497 train acc:98.29% val loss:0.2650 val acc:91.42%
12/200 train loss:0.0429 train acc:98.55% val loss:0.2763 val acc:92.44%
13/200 train loss:0.0411 train acc:98.69% val loss:0.7908 val acc:77.48%
14/200 train loss:0.0360 train acc:98.81% val loss:0.3213 va

# Task1 filling level

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
base_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/'
audio_folder = '/content/drive/MyDrive/COSRMAL_CHALLENGE/train/audio/'
os.makedirs(os.path.join(base_path, 'audios'), exist_ok=True)
mfcc_path_ = (os.path.join(base_path, 'audios', 'mfcc_t1'))
os.makedirs(mfcc_path_,exist_ok=True)

audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
save_size=64
ratio_step = 0.25
count = 0

pbar = tqdm(total=len(audio_paths))

# for i, path in enumerate(audio_paths):
#   mfcc_list = []
#   id = i
#   start_time = gt[gt.id==id]['start'].item()
#   end_time = gt[gt.id==id]['end'].item()
#   filling_level= gt[gt.id==id]['filling_level'].item()
#   sample_rate, signal = scipy.io.wavfile.read(path)


#   # TODO: Check whether it is trimed from the start
#   if id == 377:
#     signal = signal[:int(sample_rate * 13), :]
#   ap = AudioProcessing(sample_rate,signal,nfilt=save_size)
#   mfcc = ap.calc_MFCC()
#   mfcc_length=mfcc.shape[0]

#   if mfcc_length < save_size:
#     print("file {} is too short".format(id))
#   else:
#     f_step=int(mfcc.shape[1]*ratio_step)
#     f_length=mfcc.shape[1]
#     save_mfcc_num=int(np.ceil(float(np.abs(mfcc_length - save_size)) / f_step))

#     for i in range(save_mfcc_num):
#       count += 1
#       tmp_mfcc = mfcc[i*f_step:save_size+i*f_step,: ,:]

#       mfcc_list.append(tmp_mfcc)
    
#     mfcc_video = np.array(mfcc_list)
#     assert len(mfcc_video.shape) == 4, "4 channel"
#     assert mfcc_video.shape[1] +  mfcc_video.shape[2] + mfcc_video.shape[3]== 136, "136 dims"
#     np.save(os.path.join(mfcc_path_, "{0:06d}".format(id)), mfcc_video)
#   pbar.update()




  0%|          | 0/684 [00:00<?, ?it/s]

In [ ]:
print(gt['filling_type'].to_numpy().shape)
print(gt['id'].to_numpy().shape)

(684,)
(684,)


In [ ]:
class audioDataSet_t1(Dataset):
  def __init__(self,root_pth,test=False,transform = None,padding_size = 100):
    print("Dataset initializing...")
    class_num=3
    self.audio_pth = os.path.join(root_pth, 'audios', 'mfcc_t1')
    self.label = gt['filling_type'].to_numpy()
    self.is_test=test
    self.each_class_size = []
    for i in range(class_num):
        self.each_class_size.append(np.count_nonzero(self.label==i))
 
    mx=194.19187653405487
    mn=-313.07119549054045
    
    self.mn=mn
    self.mx=mx
  def __len__(self):
    return self.label.shape[0]
  def __getitem__(self, idx):
       
        lbl = self.label[idx]
        data=np.load(os.path.join(self.audio_pth, "{0:06d}".format(idx) + '.npy'), allow_pickle=True)
        data= (data-self.mn)/(self.mx-self.mn)
        data=data.transpose(0, 3, 1, 2)
        data=torch.from_numpy(data.astype(np.float32))
        return data , lbl
            
  def get_each_class_size(self):
    return np.array(self.each_class_size)

In [ ]:
class CNN_LSTM(nn.Module):
  def __init__(self, feature):
    super(CNN_LSTM, self).__init__()
    self.feature = feature
    self.proj = nn.Linear(1792, 512)
    self.lstm = nn.LSTM(input_size=512, hidden_size=256, num_layers=3, bidirectional=True)
    self.fc1 = nn.Linear(512, 128)
    self.fc2 = nn.Linear(128, 3)
  def forward(self, x_3d):
    # bs, t, c, h, w
    hidden = None
    for t in range(x_3d.size(1)):
      with torch.no_grad():
        x = self.feature.extract(x_3d[:, t, :, :, :])
      x = self.proj(x)
      out, hidden = self.lstm(x.unsqueeze(0), hidden) 

    x = self.fc1(out[-1, :, :])
    x = F.relu(x)
    x = self.fc2(x)
    return x




### Mobile

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init



class hswish(nn.Module):
    def forward(self, x):
        out = x * F.relu6(x + 3, inplace=True) / 6
        return out


class hsigmoid(nn.Module):
    def forward(self, x):
        out = F.relu6(x + 3, inplace=True) / 6
        return out


class SeModule(nn.Module):
    def __init__(self, in_size, reduction=4):
        super(SeModule, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_size, in_size // reduction, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(in_size // reduction),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_size // reduction, in_size, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(in_size),
            hsigmoid()
        )

    def forward(self, x):
        return x * self.se(x)


class Block(nn.Module):
    '''expand + depthwise + pointwise'''
    def __init__(self, kernel_size, in_size, expand_size, out_size, nolinear, semodule, stride):
        super(Block, self).__init__()
        self.stride = stride
        self.se = semodule

        self.conv1 = nn.Conv2d(in_size, expand_size, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(expand_size)
        self.nolinear1 = nolinear
        self.conv2 = nn.Conv2d(expand_size, expand_size, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, groups=expand_size, bias=False)
        self.bn2 = nn.BatchNorm2d(expand_size)
        self.nolinear2 = nolinear
        self.conv3 = nn.Conv2d(expand_size, out_size, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(out_size)

        self.shortcut = nn.Sequential()
        if stride == 1 and in_size != out_size:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_size, out_size, kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_size),
            )

    def forward(self, x):
        out = self.nolinear1(self.bn1(self.conv1(x)))
        out = self.nolinear2(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        if self.se != None:
            out = self.se(out)
        out = out + self.shortcut(x) if self.stride==1 else out
        return out


class MobileNetV3_Large(nn.Module):
    def __init__(self, num_classes=4):
        super(MobileNetV3_Large, self).__init__()
        self.conv1 = nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.hs1 = hswish()

        self.bneck = nn.Sequential(
            Block(3, 16, 16, 16, nn.ReLU(inplace=True), None, 1),
            Block(3, 16, 64, 24, nn.ReLU(inplace=True), None, 2),
            Block(3, 24, 72, 24, nn.ReLU(inplace=True), None, 1),
            Block(5, 24, 72, 40, nn.ReLU(inplace=True), SeModule(40), 2),
            Block(5, 40, 120, 40, nn.ReLU(inplace=True), SeModule(40), 1),
            Block(5, 40, 120, 40, nn.ReLU(inplace=True), SeModule(40), 1),
            Block(3, 40, 240, 80, hswish(), None, 2),
            Block(3, 80, 200, 80, hswish(), None, 1),
            Block(3, 80, 184, 80, hswish(), None, 1),
            Block(3, 80, 184, 80, hswish(), None, 1),
            Block(3, 80, 480, 112, hswish(), SeModule(112), 1),
            Block(3, 112, 672, 112, hswish(), SeModule(112), 1),
            Block(5, 112, 672, 160, hswish(), SeModule(160), 1),
            Block(5, 160, 672, 160, hswish(), SeModule(160), 2),
            Block(5, 160, 960, 160, hswish(), SeModule(160), 1),
        )


        self.conv2 = nn.Conv2d(160, 960, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(960)
        self.hs2 = hswish()
        self.linear3 = nn.Linear(960, 1280)
        self.bn3 = nn.BatchNorm1d(1280)
        self.hs3 = hswish()
        self.linear4 = nn.Linear(1280, num_classes)
        self.init_params()

    def init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.hs3(self.bn3(self.linear3(out)))
        out = self.linear4(out)
        return out
    def extract(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 2)
        return out




class MobileNetV3_Small(nn.Module):
    def __init__(self, num_classes=1000):
        super(MobileNetV3_Small, self).__init__()
        self.conv1 = nn.Conv2d(8, 16, kernel_size=3, stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.hs1 = hswish()

        self.bneck = nn.Sequential(
            Block(3, 16, 16, 16, nn.ReLU(inplace=True), SeModule(16), 2),
            Block(3, 16, 72, 24, nn.ReLU(inplace=True), None, 2),
            Block(3, 24, 88, 24, nn.ReLU(inplace=True), None, 1),
            Block(5, 24, 96, 40, hswish(), SeModule(40), 2),
            Block(5, 40, 240, 40, hswish(), SeModule(40), 1),
            Block(5, 40, 240, 40, hswish(), SeModule(40), 1),
            Block(5, 40, 120, 48, hswish(), SeModule(48), 1),
            Block(5, 48, 144, 48, hswish(), SeModule(48), 1),
            Block(5, 48, 288, 96, hswish(), SeModule(96), 2),
            Block(5, 96, 576, 96, hswish(), SeModule(96), 1),
            Block(5, 96, 576, 96, hswish(), SeModule(96), 1),
        )


        self.conv2 = nn.Conv2d(96, 576, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(576)
        self.hs2 = hswish()
        self.linear3 = nn.Linear(576, 1280)
        self.bn3 = nn.BatchNorm1d(1280)
        self.hs3 = hswish()
        self.linear4 = nn.Linear(1280, num_classes)
        self.init_params()

    def init_params(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.hs3(self.bn3(self.linear3(out)))
        out = self.linear4(out)
        return out
        
    def extract(self, x):
        out = self.hs1(self.bn1(self.conv1(x)))
        out = self.bneck(out)
        out = self.hs2(self.bn2(self.conv2(out)))
        out = F.avg_pool2d(out, 4)

        return out



def test():
    net = MobileNetV3_Small()
    x = torch.randn(2,3,224,224)
    y = net(x)
    print(y.size())

### model test

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

efficient = '/content/drive/MyDrive/COSRMAL_CHALLENGE/audios/efficient/XL-97.14.pth'


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
feature = effnetv2_xl()
feature.load_state_dict(torch.load(efficient))
model = CNN_LSTM(feature).to(device)
summary(model, input_size=(1, 2, 8, 64, 64))

Using device: cuda


Layer (type:depth-idx)                                  Output Shape              Param #
CNN_LSTM                                                --                        --
├─EffNetV2: 1                                           --                        --
│    └─Sequential: 2-1                                  [1, 640, 2, 2]            --
│    │    └─Sequential: 3-1                             [1, 24, 32, 32]           1,776
│    │    └─MBConv: 3-2                                 [1, 32, 32, 32]           6,064
│    │    └─MBConv: 3-3                                 [1, 32, 32, 32]           10,368
│    │    └─MBConv: 3-4                                 [1, 32, 32, 32]           10,368
│    │    └─MBConv: 3-5                                 [1, 32, 32, 32]           10,368
│    │    └─MBConv: 3-6                                 [1, 64, 16, 16]           45,440
│    │    └─MBConv: 3-7                                 [1, 64, 16, 16]           164,480
│    │    └─MBConv: 3-8          

In [ ]:
def train(model, train_loader, optimizer, criterion = nn.CrossEntropyLoss()):
  model.train()
  loss_train = 0.0
  correct_train = 0.0
  num_train = len(train_loader)
  for batch_idx, (audio, target) in enumerate(train_loader):
    audio = audio.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    outputs = model.forward(audio)

    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    loss_train += loss.item() / num_train
    _, preds=torch.max(outputs,1)
    correct_train+=torch.sum(preds==target).item()
  
  return loss_train, correct_train


def evaluate(model, testloader, criterion = nn.CrossEntropyLoss()):
  model.eval()
  loss_test = 0
  correct_test=0
  num_val = len(testloader)
  with torch.no_grad():
    for batch_idx, (audio, target) in enumerate(testloader):
      audio = audio.to(device)
      target = target.to(device)
      outputs = model.forward(audio)
      loss = criterion(outputs, target)
      loss_test += loss.item() / num_val
      _, preds=torch.max(outputs,1)
      correct_test+=torch.sum(preds==target).item()
  
  return loss_test, correct_test


In [ ]:
bs = 2
train_split = 0.8
lr = 1e-5
epochs = 200
mydataset_t1 = audioDataSet_t1(base_path)
n_samples = len(mydataset_t1)
assert n_samples == 684, "684"
feature_ = MobileNetV3_Large()

model = CNN_LSTM(feature_).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = int(train_split * n_samples)
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(mydataset_t1, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=True)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("Epoch {}/{} train loss:{:.4f} train acc:{:.2f}% ".format(epoch+1,epochs, loss_train, 100 * correct_train/num_train))
  print("Epoch {}/{} val loss:{:.4f} val acc:{:.2f}% ".format(epoch+1,epochs, loss_val, 100 * correct_val/num_val))

  if loss_val < best_loss:
    best_loss = loss_val
    torch.save(model, os.path.join(base_path, 'audios', "best_loss.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    torch.save(model, os.path.join(base_path, 'audios', "best_val.pth"))
  




# Task1 filling level audio



In [ ]:
efficient = '/content/drive/MyDrive/COSRMAL_CHALLENGE/audios/efficient/XL-97.14.pth'
base_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE/'
audio_folder = '/content/drive/MyDrive/COSRMAL_CHALLENGE/train/audio'
T2_mid_dir = os.path.join(base_path, 'T2_mid')
T2_pred_dir = os.path.join(base_path, 'T2_pred')
os.makedirs(T2_mid_dir,exist_ok=True)
os.makedirs(T2_pred_dir,exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
model = effnetv2_xl()
model.load_state_dict(torch.load(efficient))
model.to(device)
model.eval()

audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
save_size=64
ratio_step = 0.25
count = 0
MAX_VALUE=194.19187653405487
MIN_VALUE=-313.07119549054045

# pbar = tqdm(total=len(audio_paths))

# for i, path in enumerate(audio_paths):
#   id = i
#   start_time = gt[gt.id==id]['start'].item()
#   end_time = gt[gt.id==id]['end'].item()
#   filling_type = gt[gt.id==id]['filling_type'].item()
#   datalist = []
#   predlist = []
#   sample_rate, signal = scipy.io.wavfile.read(path)
#   ap = AudioProcessing(sample_rate,signal,nfilt=save_size)
#   mfcc = ap.calc_MFCC()
#   mfcc_length=mfcc.shape[0]

#   if mfcc_length < save_size:
#     print("file {} is too short".format(id))
#   else:
#     f_step=int(mfcc.shape[1]*ratio_step)
#     f_length=mfcc.shape[1]
#     save_mfcc_num=int(np.ceil(float(np.abs(mfcc_length - save_size)) / f_step))

#     for i in range(save_mfcc_num):
#       tmp_mfcc = mfcc[i*f_step:save_size+i*f_step,: ,:]
#       tmp_mfcc= (tmp_mfcc-MIN_VALUE)/(MAX_VALUE-MIN_VALUE)
#       tmp_mfcc=tmp_mfcc.transpose(2,0,1)
#       audio=torch.from_numpy(tmp_mfcc.astype(np.float32))
#       audio=torch.unsqueeze(audio, 0)
#       audio = audio.to(device) 
#       feature, pred=model.extract(audio)
#       _,pred=torch.max(pred,1)
#       datalist.append(feature.to('cpu').detach().numpy().copy())
#       predlist.append(pred.item())
#     datalist = np.squeeze(np.array(datalist))
#     predlist = np.squeeze(np.array(predlist))
#     np.save(os.path.join(T2_mid_dir, "{0:06d}".format(id)), datalist)
#     np.save(os.path.join(T2_pred_dir, "{0:06d}".format(id)), predlist)

#   pbar.update()



Using device: cuda


  0%|          | 0/684 [00:00<?, ?it/s]

In [ ]:
class Padding(object):
    def __init__(self, seq_len):
        self.seq_len = seq_len

    def __call__(self, sample, pred):
        #np.clip(pred, 0,1,out=pred)
        sample_len, input_dim = sample.shape
        #for i in range(sample_len):
        #    sample[i, :] *= pred[i]

        if sample_len >= self.seq_len:
            features = sample[:self.seq_len, :]
            return features
        else:
            start_seq = np.random.randint(0, self.seq_len - sample_len+1)
            #ini=[1]+[0]*(input_dim-1)
            ini=[0]*(input_dim)
            features = np.full((self.seq_len, input_dim),ini, dtype = float)
            features[start_seq:start_seq+sample_len, :] = sample
            return features

class MyLSTMDataset(torch.utils.data.Dataset):
    def __init__(self,root_pth,test=False,transform = None, padding_size = 100):
        class_num=3
        self.mid_pth = os.path.join(root_pth, 'T2_mid')
        self.pred_pth = os.path.join(root_pth, 'T2_pred')
        self.label = gt['filling_level'].to_numpy()
        self.is_test=test
        self.each_class_size = []
        self.each_class_sum = [0]*class_num
        for i in range(class_num):
            self.each_class_size.append(np.count_nonzero(self.label==i))
        mx=0
        mn=1000
        len_mx = 0
        
        for idx in range(self.label.shape[0]):
            data=np.load(os.path.join(self.mid_pth, "{0:06d}".format(idx) + '.npy'), allow_pickle=True)
            self.each_class_sum[self.label[idx]]+=data.shape[0]
            if data.shape[0] > len_mx:
                len_mx=data.shape[0]
            tmp_max=np.max(data)
            tmp_min=np.min(data)
            if mx<tmp_max:
                mx=tmp_max
            if mn>tmp_min:
                mn=tmp_min
        self.mn=mn
        self.mx=mx
        self.pad = Padding(padding_size)
        print(len_mx)
            
    def __len__(self):
        return self.label.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        lbl = -1

        if self.is_test is False:
            lbl = self.label[idx]
            
        data=np.load(os.path.join(self.mid_pth, "{0:06d}".format(idx) + '.npy'), allow_pickle=True)
        pred=np.load(os.path.join(self.pred_pth, "{0:06d}".format(idx) + '.npy'), allow_pickle=True)
        data = (data-self.mn)/(self.mx-self.mn)
        data = self.pad(data, pred)

        #np.clip(data, 0,1,out=data)
        data=torch.from_numpy(data.astype(np.float32))
        return data , lbl
            
    def get_each_class_size(self):
        return np.array(self.each_class_size)

    def get_each_class_avg_len(self):
        each_class_avg_len =  np.array(self.each_class_sum)/np.array(self.each_class_size)
        all_class_avg_len = np.sum(np.array(self.each_class_sum))/np.sum(np.array(self.each_class_size))
        return each_class_avg_len, all_class_avg_len

In [ ]:
base_path = '/content/drive/MyDrive/COSRMAL_CHALLENGE'
myDataSet = MyLSTMDataset(base_path)

953


In [ ]:
class CNN_LSTM(nn.Module):
  def __init__(self):
    super(CNN_LSTM, self).__init__()
    #self.proj = nn.Linear(1792, 512)
    self.lstm = nn.LSTM(input_size=1792, hidden_size=256, num_layers=3, dropout=0.15, bidirectional=True)
    self.fc1 = nn.Linear(256, 128)
    self.fc2 = nn.Linear(128, 3)
  
  def attnetwork(self, encoder_out, final_hidden):
    hidden = final_hidden.squeeze(0)
    #M = torch.tanh(encoder_out)
    attn_weights = torch.bmm(encoder_out, hidden.unsqueeze(2)).squeeze(2)
    soft_attn_weights = F.softmax(attn_weights, 1)
    new_hidden = torch.bmm(encoder_out.transpose(1,2), soft_attn_weights.unsqueeze(2)).squeeze(2)
    #print (wt.shape, new_hidden.shape)
    #new_hidden = torch.tanh(new_hidden)
    #print ('UP:', new_hidden, new_hidden.shape)
        
    return new_hidden
  def forward(self, x_3d):
    # bs, t, c, h, w
    hidden = None
    for t in range(x_3d.size(1)):
      #x = self.proj(x_3d[:, t])
      output, hidden = self.lstm(x_3d[:, t].unsqueeze(0), hidden) 
    

    (hn, cn) = hidden
    fbout = output[:, :, :256]+ output[:, :, 256:] #sum bidir outputs F+B
    fbout = fbout.permute(1,0,2)
    fbhn = (hn[-2,:,:]+hn[-1,:,:]).unsqueeze(0)
    #print (fbhn.shape, fbout.shape)
    attn_out = self.attnetwork(fbout, fbhn)

    #x = self.fc1(out[-1, :, :])
    x = self.fc1(attn_out)
    x = F.relu(x)
    x = self.fc2(x)
    return x




In [ ]:
def train(model, train_loader, optimizer, criterion = nn.CrossEntropyLoss()):
  model.train()
  loss_train = 0.0
  correct_train = 0.0
  num_train = len(train_loader)
  for batch_idx, (audio, target) in enumerate(train_loader):
    audio = audio.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    outputs = model.forward(audio)

    loss = criterion(outputs, target)
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    loss_train += loss.item() / num_train
    _, preds=torch.max(outputs,1)
    correct_train+=torch.sum(preds==target).item()
  
  return loss_train, correct_train


def evaluate(model, testloader, criterion = nn.CrossEntropyLoss()):
  model.eval()
  loss_test = 0
  correct_test=0
  num_val = len(testloader)
  with torch.no_grad():
    for batch_idx, (audio, target) in enumerate(testloader):
      audio = audio.to(device)
      target = target.to(device)
      outputs = model.forward(audio)
      loss = criterion(outputs, target)
      loss_test += loss.item() / num_val
      _, preds=torch.max(outputs,1)
      correct_test+=torch.sum(preds==target).item()
  
  return loss_test, correct_test


In [ ]:
bs = 16
train_split = 0.8
lr = 1e-4
epochs = 200
n_samples = len(myDataSet)
assert n_samples == 684, "684"

model = CNN_LSTM().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = 584
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(myDataSet, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=False)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("Epoch {}/{} train loss:{:.4f} train acc:{:.2f}% ".format(epoch+1,epochs, loss_train, 100 * correct_train/num_train))
  print("Epoch {}/{} val loss:{:.4f} val acc:{:.2f}% ".format(epoch+1,epochs, loss_val, 100 * correct_val/num_val))

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "best_loss.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    best_train = correct_train
    torch.save(model, os.path.join(base_path, 'audios', "best_lstm.pth"))
  
  if correct_val == best_acc and best_train < correct_train:
    best_acc = correct_val
    best_train = correct_train
    torch.save(model, os.path.join(base_path, 'audios', "best_lstm.pth"))

    
  




Epoch 1/200 train loss:1.0452 train acc:46.40% 
Epoch 1/200 val loss:0.9191 val acc:41.00% 
Epoch 2/200 train loss:0.9919 train acc:50.00% 
Epoch 2/200 val loss:0.9260 val acc:61.00% 
Epoch 3/200 train loss:0.9881 train acc:49.14% 
Epoch 3/200 val loss:0.8794 val acc:58.00% 
Epoch 4/200 train loss:0.9932 train acc:51.54% 
Epoch 4/200 val loss:0.8525 val acc:66.00% 
Epoch 5/200 train loss:0.9659 train acc:49.66% 
Epoch 5/200 val loss:0.8356 val acc:60.00% 
Epoch 6/200 train loss:0.8339 train acc:46.92% 
Epoch 6/200 val loss:0.7376 val acc:67.00% 
Epoch 7/200 train loss:0.7594 train acc:64.21% 
Epoch 7/200 val loss:0.6884 val acc:63.00% 
Epoch 8/200 train loss:0.6278 train acc:68.32% 
Epoch 8/200 val loss:0.5749 val acc:76.00% 
Epoch 9/200 train loss:0.6022 train acc:63.36% 
Epoch 9/200 val loss:0.5800 val acc:73.00% 
Epoch 10/200 train loss:0.5727 train acc:64.38% 
Epoch 10/200 val loss:0.5843 val acc:73.00% 
Epoch 11/200 train loss:0.5532 train acc:64.73% 
Epoch 11/200 val loss:0.5212 

In [ ]:
bs = 16
train_split = 0.8
lr = 1e-4
epochs = 200
n_samples = len(myDataSet)
assert n_samples == 684, "684"

model = CNN_LSTM().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr,  weight_decay=1e-5)

best_loss = float('inf')
best_acc = 0

num_train = 584
num_val = n_samples - num_train

train_set, val_set = torch.utils.data.random_split(myDataSet, [num_train, num_val])

assert len(train_set) == num_train, "Same"
assert len(val_set) == num_val, "Same"


train_loader   = DataLoader(train_set,
                            batch_size=bs,
                            shuffle=True)
val_loader   = DataLoader(val_set,
                          batch_size=bs,
                          shuffle=False)

for epoch in range(epochs):
  #start_time = time.time()
  loss_train, correct_train = train(model, train_loader, optimizer)
  loss_val, correct_val = evaluate(model, val_loader, criterion = nn.CrossEntropyLoss())
  #elapsed_time = time.time() - start_time
  print("Epoch {}/{} train loss:{:.4f} train acc:{:.2f}% ".format(epoch+1,epochs, loss_train, 100 * correct_train/num_train))
  print("Epoch {}/{} val loss:{:.4f} val acc:{:.2f}% ".format(epoch+1,epochs, loss_val, 100 * correct_val/num_val))

  # if loss_val < best_loss:
  #   best_loss = loss_val
  #   torch.save(model, os.path.join(base_path, 'audios', "best_loss.pth"))
  
  if correct_val > best_acc:
    best_acc = correct_val
    best_train = correct_train
    torch.save(model, os.path.join(base_path, 'audios', "best_lstm_att.pth"))
  
  if correct_val == best_acc and best_train < correct_train:
    best_acc = correct_val
    best_train = correct_train
    torch.save(model, os.path.join(base_path, 'audios', "best_lstm_att.pth"))

    
  




Epoch 1/200 train loss:1.0357 train acc:42.64% 
Epoch 1/200 val loss:0.9610 val acc:35.00% 
Epoch 2/200 train loss:1.0011 train acc:51.54% 
Epoch 2/200 val loss:1.0445 val acc:59.00% 
Epoch 3/200 train loss:0.9815 train acc:48.97% 
Epoch 3/200 val loss:1.0195 val acc:54.00% 
Epoch 4/200 train loss:0.9543 train acc:55.65% 
Epoch 4/200 val loss:0.9913 val acc:57.00% 
Epoch 5/200 train loss:0.9740 train acc:49.83% 
Epoch 5/200 val loss:0.9485 val acc:58.00% 
Epoch 6/200 train loss:0.8934 train acc:51.88% 
Epoch 6/200 val loss:0.7982 val acc:51.00% 
Epoch 7/200 train loss:0.7878 train acc:55.14% 
Epoch 7/200 val loss:0.6595 val acc:74.00% 
Epoch 8/200 train loss:0.7241 train acc:61.47% 
Epoch 8/200 val loss:0.6126 val acc:70.00% 
Epoch 9/200 train loss:0.5849 train acc:67.64% 
Epoch 9/200 val loss:0.6113 val acc:68.00% 
Epoch 10/200 train loss:0.6306 train acc:64.04% 
Epoch 10/200 val loss:0.5440 val acc:67.00% 
Epoch 11/200 train loss:0.5608 train acc:66.95% 
Epoch 11/200 val loss:0.5514 